# Реализация в Pose2Sim калибровки камер

In [ ]:
# Клонируем репозиторий pose2sim
!git clone --depth 1 https://github.com/perfanalytics/pose2sim.git

Cloning into 'pose2sim'...
remote: Enumerating objects: 406, done.
remote: Counting objects: 100% (406/406), done.
remote: Compressing objects: 100% (389/389), done.
remote: Total 406 (delta 42), reused 332 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (406/406), 37.99 MiB | 25.41 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [ ]:
!pip install mpl_interactions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.0 MB/s eta 0:00:00


In [ ]:
import os
import logging
import pickle
import numpy as np
import pandas as pd
os.environ["OPENCV_LOG_LEVEL"]="FATAL"
import cv2
import glob
import toml
import re
from lxml import etree
import warnings
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory, panhandler
from PIL import Image
from contextlib import contextmanager,redirect_stderr,redirect_stdout
from os import devnull
import time
from datetime import datetime

In [ ]:
#@title Реализация в Pose2Sim получение уровня и списка конфигурационных словарей
import os
import toml
from copy import deepcopy

config_dir = '/content/pose2sim/Pose2Sim/Demo_SinglePerson/'

project_dir = '/content/pose2sim/Pose2Sim/Demo_SinglePerson/'

def recursive_update(dict_to_update, dict_with_new_values):
    '''
    Обновляет вложенные словари, не перезаписывая существующие ключи на любом уровне вложенности.

    Пример:
    dict_to_update = {'key': {'key_1': 'val_1', 'key_2': 'val_2'}}
    dict_with_new_values = {'key': {'key_1': 'val_1_new'}}
    возвращает {'key': {'key_1': 'val_1_new', 'key_2': 'val_2'}}
    в то время как dict_to_update.update(dict_with_new_values) вернет {'key': {'key_1': 'val_1_new'}}
    '''

    # Проходим по всем ключам и значениям в словаре с новыми значениями
    for key, value in dict_with_new_values.items():
        # Проверяем, существует ли ключ в словаре для обновления и является ли значение словарем
        if key in dict_to_update and isinstance(value, dict) and isinstance(dict_to_update[key], dict):
            # Рекурсивно обновляем вложенные словари
            dict_to_update[key] = recursive_update(dict_to_update[key], value)
        else:
            # Обновляем или добавляем новые пары ключ-значение
            dict_to_update[key] = value

    # Возвращаем обновленный словарь
    return dict_to_update

def determine_level(config_dir):
    '''
    Определяет уровень, на котором вызывается функция.
    Уровень = 1: Папка испытания (Trial folder)
    Уровень = 2: Корневая папка (Root folder)
    '''

    # Создаем список, в котором храним длину пути для каждой папки,
    # содержащей файл 'Config.toml'. Длина пути определяется количеством
    # разделителей в пути (os.sep).
    len_paths = [len(root.split(os.sep)) for root, dirs, files in os.walk(config_dir) if 'Config.toml' in files]

    # Если список len_paths пуст, это означает, что не найдено ни одного файла 'Config.toml'.
    # В этом случае выбрасываем исключение FileNotFoundError с соответствующим сообщением.
    if len_paths == []:
        raise FileNotFoundError('Вам нужен файл Config.toml в каждой папке испытания или корневой папке.')

    # Определяем уровень, вычисляя разницу между максимальной и минимальной длиной пути,
    # добавляя 1, чтобы учесть уровень.
    level = max(len_paths) - min(len_paths) + 1

    # Возвращаем определенный уровень.
    return level


def read_config_files(config=None, config_dir='/content/pose2sim/Pose2Sim/Demo_SinglePerson/'):
    '''
    Читает корневые и пробные конфигурационные файлы
    и возвращает словарь со всеми параметрами.
    '''

    if type(config) == dict:
        level = 2  # Уровень 2, логическая директория = текущая рабочая директория
        config_dicts = [config]
        if config_dicts[0].get('project').get('project_dir') is None:
            raise ValueError('Пожалуйста, укажите директорию проекта в config_dict:\n \
                             config_dict.get("project").update({"project_dir":"<ВАША_ДИРЕКТОРИЯ_ПРОЕКТА>"})')
    else:
        # Если вызван без аргумента, config == None, иначе это путь к директории конфигурации
        config_dir = config_dir if config is None else config  # Используем config_dir
        level = determine_level(config_dir)  # Определяем уровень конфигурации

        # Уровень пробного испытания
        if level == 1:  # Уровень пробного испытания
            try:
                # Если пакетная обработка
                session_config_dict = toml.load(os.path.join(config_dir, '..', 'Config.toml'))
                trial_config_dict = toml.load(os.path.join(config_dir, 'Config.toml'))
                session_config_dict = recursive_update(session_config_dict, trial_config_dict)  # Обновляем словарь сессии
            except:
                # Если одно пробное испытание
                session_config_dict = toml.load(os.path.join(config_dir, 'Config.toml'))
            session_config_dict.get("project").update({"project_dir": config_dir})  # Устанавливаем директорию проекта
            config_dicts = [session_config_dict]  # Сохраняем конфигурацию сессии в список

        # Корневой уровень
        if level == 2:
            session_config_dict = toml.load(os.path.join(config_dir, 'Config.toml'))  # Загружаем корневую конфигурацию
            config_dicts = []
            # Создаем конфигурационные словари для всех пробных испытаний участника
            for (root, dirs, files) in os.walk(config_dir):
                if 'Config.toml' in files and root != config_dir:
                    trial_config_dict = toml.load(os.path.join(root, 'Config.toml'))  # Загружаем конфигурацию пробного испытания
                    # Глубокое копирование, иначе session_config_dict изменяется на каждой итерации в списке config_dicts
                    temp_dict = deepcopy(session_config_dict)
                    temp_dict = recursive_update(temp_dict, trial_config_dict)  # Обновляем временный словарь
                    temp_dict.get("project").update({"project_dir": os.path.join(config_dir, os.path.relpath(root))})  # Устанавливаем директорию проекта для пробного испытания
                    if not os.path.basename(root) in temp_dict.get("project").get('exclude_from_batch'):
                        config_dicts.append(temp_dict)  # Добавляем временный словарь в список конфигураций

    return level, config_dicts  # Возвращаем уровень и список конфигурационных словарей


level, config_dicts = read_config_files(config=None, config_dir=config_dir)
config_dict = config_dicts[0]


In [ ]:
config_dict

{'project': {'multi_person': False,
  'participant_height': 1.72,
  'participant_mass': 70.0,
  'frame_rate': 'auto',
  'frame_range': [],
  'exclude_from_batch': [],
  'project_dir': '/content/pose2sim/Pose2Sim/Demo_SinglePerson/'},
 'pose': {'vid_img_extension': 'mp4',
  'pose_model': 'HALPE_26',
  'mode': 'balanced',
  'det_frequency': 1,
  'display_detection': True,
  'overwrite_pose': False,
  'save_video': 'to_video',
  'output_format': 'openpose',
  'CUSTOM': {'name': 'Hip',
   'id': '19',
   'children': [{'name': 'RHip',
     'id': 12,
     'children': [{'name': 'RKnee',
       'id': 14,
       'children': [{'name': 'RAnkle',
         'id': 16,
         'children': [{'name': 'RBigToe',
           'id': 21,
           'children': [{'name': 'RSmallToe', 'id': 23}]},
          {'name': 'RHeel', 'id': 25}]}]}]},
    {'name': 'LHip',
     'id': 11,
     'children': [{'name': 'LKnee',
       'id': 13,
       'children': [{'name': 'LAnkle',
         'id': 15,
         'children': [{'n

In [ ]:
level

1

In [ ]:
config_dict.keys()

dict_keys(['project', 'pose', 'synchronization', 'calibration', 'personAssociation', 'triangulation', 'filtering', 'markerAugmentation', 'kinematics'])

## Калибровка камер
В конфиге изменим настройки
[calibration]
calibration_type = 'calculate'

In [ ]:
def imgp_objp_visualizer_clicker(img, imgp=[], objp=[], img_path=''):
    '''
    Shows image img.
    If imgp is given, displays them in green
    If objp is given, can be displayed in a 3D plot if 'C' is pressed.
    If img_path is given, just uses it to name the window

    If 'Y' is pressed, closes all and returns confirmed imgp and (if given) objp
    If 'N' is pressed, closes all and returns nothing
    If 'C' is pressed, allows clicking imgp by hand. If objp is given:
        Displays them in 3D as a helper.
        Left click to add a point, right click to remove the last point.
        Press 'H' to indicate that one of the objp is not visible on image
        Closes all and returns imgp and objp if all points have been clicked
    Allows for zooming and panning with middle click

    INPUTS:
    - img: image opened with openCV
    - optional: imgp: detected image points, to be accepted or not. Array of [[2d corner coordinates]]
    - optional: objp: array of [3d corner coordinates]
    - optional: img_path: path to image

    OUTPUTS:
    - imgp_confirmed: image points that have been correctly identified. array of [[2d corner coordinates]]
    - only if objp!=[]: objp_confirmed: array of [3d corner coordinates]
    '''
    global old_image_path
    old_image_path = img_path

    def on_key(event):
        '''
        Handles key press events:
        'Y' to return imgp, 'N' to dismiss image, 'C' to click points by hand.
        Left click to add a point, 'H' to indicate it is not visible, right click to remove the last point.
        '''

        global imgp_confirmed, objp_confirmed, objp_confirmed_notok, scat, ax_3d, fig_3d, events, count

        if event.key == 'y':
            # If 'y', close all
            # If points have been clicked, imgp_confirmed is returned, else imgp
            # If objp is given, objp_confirmed is returned in addition
            if 'scat' not in globals() or 'imgp_confirmed' not in globals():
                imgp_confirmed = imgp
                objp_confirmed = objp
            else:
                imgp_confirmed = np.array([imgp.astype('float32') for imgp in imgp_confirmed])
                objp_confirmed = objp_confirmed
            # OpenCV needs at leas 4 correspondance points to calibrate
            if len(imgp_confirmed) < 6:
                objp_confirmed = []
                imgp_confirmed = []
            # close all, del all global variables except imgp_confirmed and objp_confirmed
            plt.close('all')
            if len(objp) == 0:
                if 'objp_confirmed' in globals():
                    del objp_confirmed

        if event.key == 'n' or event.key == 'q':
            # If 'n', close all and return nothing
            plt.close('all')
            imgp_confirmed = []
            objp_confirmed = []

        if event.key == 'c':
            # TODO: RIGHT NOW, IF 'C' IS PRESSED ANOTHER TIME, OBJP_CONFIRMED AND IMGP_CONFIRMED ARE RESET TO []
            # We should reopen a figure without point on it
            img_for_pointing = cv2.imread(old_image_path)
            if img_for_pointing is None:
                cap = cv2.VideoCapture(old_image_path)
                ret, img_for_pointing = cap.read()
            img_for_pointing = cv2.cvtColor(img_for_pointing, cv2.COLOR_BGR2RGB)
            ax.imshow(img_for_pointing)
            # To update the image
            plt.draw()

            if 'objp_confirmed' in globals():
                del objp_confirmed
            # If 'c', allows retrieving imgp_confirmed by clicking them on the image
            scat = ax.scatter([],[],s=100,marker='+',color='g')
            plt.connect('button_press_event', on_click)
            # If objp is given, display 3D object points in black
            if len(objp) != 0 and not plt.fignum_exists(2):
                fig_3d = plt.figure()
                fig_3d.tight_layout()
                fig_3d.canvas.manager.set_window_title('Object points to be clicked')
                ax_3d = fig_3d.add_subplot(projection='3d')
                plt.rc('xtick', labelsize=5)
                plt.rc('ytick', labelsize=5)
                for i, (xs,ys,zs) in enumerate(np.float32(objp)):
                    ax_3d.scatter(xs,ys,zs, marker='.', color='k')
                    ax_3d.text(xs,ys,zs,  f'{str(i+1)}', size=10, zorder=1, color='k')
                set_axes_equal(ax_3d)
                ax_3d.set_xlabel('X')
                ax_3d.set_ylabel('Y')
                ax_3d.set_zlabel('Z')
                if np.all(objp[:,2] == 0):
                    ax_3d.view_init(elev=-90, azim=0)
                fig_3d.show()

        if event.key == 'h':
            # If 'h', indicates that one of the objp is not visible on image
            # Displays it in red on 3D plot
            if len(objp) != 0  and 'ax_3d' in globals():
                count = [0 if 'count' not in globals() else count+1][0]
                if 'events' not in globals():
                    # retrieve first objp_confirmed_notok and plot 3D
                    events = [event]
                    objp_confirmed_notok = objp[count]
                    ax_3d.scatter(*objp_confirmed_notok, marker='o', color='r')
                    fig_3d.canvas.draw()
                elif count == len(objp)-1:
                    # if all objp have been clicked or indicated as not visible, close all
                    objp_confirmed = np.array([[objp[count]] if 'objp_confirmed' not in globals() else objp_confirmed+[objp[count]]][0])[:-1]
                    imgp_confirmed = np.array(np.expand_dims(scat.get_offsets(), axis=1), np.float32)
                    plt.close('all')
                    for var_to_delete in ['events', 'count', 'scat', 'fig_3d', 'ax_3d', 'objp_confirmed_notok']:
                        if var_to_delete in globals():
                            del globals()[var_to_delete]
                else:
                    # retrieve other objp_confirmed_notok and plot 3D
                    events.append(event)
                    objp_confirmed_notok = objp[count]
                    ax_3d.scatter(*objp_confirmed_notok, marker='o', color='r')
                    fig_3d.canvas.draw()
            else:
                pass


    def on_click(event):
        '''
        Detect click position on image
        If right click, last point is removed
        '''

        global imgp_confirmed, objp_confirmed, objp_confirmed_notok, scat, ax_3d, fig_3d, events, count, xydata

        # Left click: Add clicked point to imgp_confirmed
        # Display it on image and on 3D plot
        if event.button == 1:
            # To remember the event to cancel after right click
            if 'events' in globals():
                events.append(event)
            else:
                events = [event]

            # Add clicked point to image
            xydata = scat.get_offsets()
            new_xydata = np.concatenate((xydata,[[event.xdata,event.ydata]]))
            scat.set_offsets(new_xydata)
            imgp_confirmed = np.expand_dims(scat.get_offsets(), axis=1)
            plt.draw()

            # Add clicked point to 3D object points if given
            if len(objp) != 0:
                count = [0 if 'count' not in globals() else count+1][0]
                if count==0:
                    # retrieve objp_confirmed and plot 3D
                    objp_confirmed = [objp[count]]
                    ax_3d.scatter(*objp[count], marker='o', color='g')
                    fig_3d.canvas.draw()
                elif count == len(objp)-1:
                    # close all
                    plt.close('all')
                    # retrieve objp_confirmed
                    objp_confirmed = np.array([[objp[count]] if 'objp_confirmed' not in globals() else objp_confirmed+[objp[count]]][0])
                    imgp_confirmed = np.array(imgp_confirmed, np.float32)
                    # delete all
                    for var_to_delete in ['events', 'count', 'scat', 'scat_3d', 'fig_3d', 'ax_3d', 'objp_confirmed_notok']:
                        if var_to_delete in globals():
                            del globals()[var_to_delete]
                else:
                    # retrieve objp_confirmed and plot 3D
                    objp_confirmed = [[objp[count]] if 'objp_confirmed' not in globals() else objp_confirmed+[objp[count]]][0]
                    ax_3d.scatter(*objp[count], marker='o', color='g')
                    fig_3d.canvas.draw()


        # Right click:
        # If last event was left click, remove last point and if objp given, from objp_confirmed
        # If last event was 'H' and objp given, remove last point from objp_confirmed_notok
        elif event.button == 3: # right click
            if 'events' in globals():
                # If last event was left click:
                if 'button' in dir(events[-1]):
                    if events[-1].button == 1:
                        # Remove lastpoint from image
                        new_xydata = scat.get_offsets()[:-1]
                        scat.set_offsets(new_xydata)
                        plt.draw()
                        # Remove last point from imgp_confirmed
                        imgp_confirmed = imgp_confirmed[:-1]
                        if len(objp) != 0:
                            if count >= 0:
                                count -= 1
                            # Remove last point from objp_confirmed
                            objp_confirmed = objp_confirmed[:-1]
                            # remove from plot
                            if len(ax_3d.collections) > len(objp):
                                ax_3d.collections[-1].remove()
                                fig_3d.canvas.draw()

                # If last event was 'h' key
                elif events[-1].key == 'h':
                    if len(objp) != 0:
                        if count >= 1: count -= 1
                        # Remove last point from objp_confirmed_notok
                        objp_confirmed_notok = objp_confirmed_notok[:-1]
                        # remove from plot
                        if len(ax_3d.collections) > len(objp):
                            ax_3d.collections[-1].remove()
                            fig_3d.canvas.draw()


    def set_axes_equal(ax):
        '''
        Make axes of 3D plot have equal scale so that spheres appear as spheres,
        cubes as cubes, etc.
        From https://stackoverflow.com/questions/13685386/how-to-set-the-equal-aspect-ratio-for-all-axes-x-y-z

        Input
        ax: a matplotlib axis, e.g., as output from plt.gca().
        '''

        x_limits = ax.get_xlim3d()
        y_limits = ax.get_ylim3d()
        z_limits = ax.get_zlim3d()

        x_range = abs(x_limits[1] - x_limits[0])
        x_middle = np.mean(x_limits)
        y_range = abs(y_limits[1] - y_limits[0])
        y_middle = np.mean(y_limits)
        z_range = abs(z_limits[1] - z_limits[0])
        z_middle = np.mean(z_limits)

        # The plot bounding box is a sphere in the sense of the infinity
        # norm, hence I call half the max range the plot radius.
        plot_radius = 0.5*max([x_range, y_range, z_range])

        ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
        ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
        ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

    # Write instructions
    cv2.putText(img, 'Type "Y" to accept point detection.', (20, 20), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
    cv2.putText(img, 'Type "Y" to accept point detection.', (20, 20), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)
    cv2.putText(img, 'If points are wrongfully (or not) detected:', (20, 43), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
    cv2.putText(img, 'If points are wrongfully (or not) detected:', (20, 43), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)
    cv2.putText(img, '- type "N" to dismiss this image,', (20, 66), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
    cv2.putText(img, '- type "N" to dismiss this image,', (20, 66), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)
    cv2.putText(img, '- type "C" to click points by hand (beware of their order).', (20, 89), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
    cv2.putText(img, '- type "C" to click points by hand (beware of their order).', (20, 89), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)
    cv2.putText(img, '   left click to add a point, right click to remove it, "H" to indicate it is not visible. ', (20, 112), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
    cv2.putText(img, '   left click to add a point, right click to remove it, "H" to indicate it is not visible. ', (20, 112), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)
    cv2.putText(img, '   Confirm with "Y", cancel with "N".', (20, 135), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
    cv2.putText(img, '   Confirm with "Y", cancel with "N".', (20, 135), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)
    cv2.putText(img, 'Use mouse wheel to zoom in and out and to pan', (20, 158), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
    cv2.putText(img, 'Use mouse wheel to zoom in and out and to pan', (20, 158), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)

    # Put image in a matplotlib figure for more controls
    plt.rcParams['toolbar'] = 'None'
    fig, ax = plt.subplots()
    fig = plt.gcf()
    fig.canvas.manager.set_window_title(os.path.basename(img_path))
    ax.axis("off")
    for corner in imgp:
        x, y = corner.ravel()
        cv2.drawMarker(img, (int(x),int(y)), (128,128,128), cv2.MARKER_CROSS, 10, 2)
    ax.imshow(img)

    #Закоментируем
    #figManager = plt.get_current_fig_manager()
    #figManager.window.showMaximized()

    plt.tight_layout()

    # Allow for zoom and pan in image
    zoom_factory(ax)
    ph = panhandler(fig, button=2)

    # Handles key presses to Accept, dismiss, or click points by hand
    cid = fig.canvas.mpl_connect('key_press_event', on_key)

    plt.draw()
    plt.show(block=True)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        plt.rcParams['toolbar'] = 'toolmanager'

    for var_to_delete in ['events', 'count', 'scat', 'fig_3d', 'ax_3d', 'objp_confirmed_notok']:
        if var_to_delete in globals():
            del globals()[var_to_delete]

    if 'imgp_confirmed' in globals() and 'objp_confirmed' in globals():
        return imgp_confirmed, objp_confirmed
    elif 'imgp_confirmed' in globals() and not 'objp_confirmed' in globals():
        return imgp_confirmed
    else:
        return

In [ ]:
def findCorners(img_path, corner_nb, objp=[], show=True):
    '''
    Найти углы на фотографии шахматной доски.
    Нажмите 'Y', чтобы подтвердить обнаружение, 'N', чтобы отклонить это изображение, 'C', чтобы щелкнуть точки вручную.
    Левый клик для добавления точки, правый клик для удаления последней точки.
    Используйте колесо мыши для увеличения и уменьшения масштаба, а также для панорамирования.

    Убедитесь, что:
    - шахматная доска окружена белой рамкой
    - количество рядов не равно количеству линий, и ряд четный, если линии нечетные (или наоборот)
    - она плоская и без отражений
    - corner_nb соответствует _внутренним_ углам

    ВХОДНЫЕ ДАННЫЕ:
    - img_path: путь к изображению (или видео)
    - corner_nb: [H, W] внутренние углы на шахматной доске: список из двух целых чисел [4,7]
    - необязательный: show: выберите, показывать ли обнаруженные углы
    - необязательный: objp: массив [3d координаты углов]

    ВЫХОДНЫЕ ДАННЫЕ:
    - imgp_confirmed: массив [[2d координаты углов]]
    - только если objp!=[]: objp_confirmed: массив [3d координаты углов]
    '''

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) # остановить уточнение после 30 итераций или если ошибка меньше 0.001 пикселя

    img = cv2.imread(img_path)  # Чтение изображения из указанного пути
    if img is None:  # Если изображение не найдено, попытка прочитать из видео
        cap = cv2.VideoCapture(img_path)
        ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Преобразование изображения в оттенки серого
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Преобразование изображения в формат RGB

    # Поиск углов
    ret, corners = cv2.findChessboardCorners(gray, corner_nb, None)
    # Если углы найдены, уточнить их положение
    if ret == True:
        imgp = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)  # Уточнение углов
        logging.info(f'{os.path.basename(img_path)}: Углы найдены.')

        if show:
            # Отрисовка углов на изображении
            cv2.drawChessboardCorners(img, corner_nb, imgp, ret)
            # Добавление индекса угла
            for i, corner in enumerate(imgp):
                if i in [0, corner_nb[0]-1, corner_nb[0]*(corner_nb[1]-1), corner_nb[0]*corner_nb[1] -1]:
                    x, y = corner.ravel()
                    cv2.putText(img, str(i+1), (int(x)-5, int(y)-5), cv2.FONT_HERSHEY_SIMPLEX, .8, (255, 255, 255), 7)
                    cv2.putText(img, str(i+1), (int(x)-5, int(y)-5), cv2.FONT_HERSHEY_SIMPLEX, .8, (0,0,0), 2)

            # Визуализатор и обработчик событий нажатия клавиш
            for var_to_delete in ['imgp_confirmed', 'objp_confirmed']:
                if var_to_delete in globals():
                    del globals()[var_to_delete]
            imgp_objp_confirmed = imgp_objp_visualizer_clicker(img, imgp=imgp, objp=objp, img_path=img_path)
        else:
            imgp_objp_confirmed = imgp  # Если не показывать, просто возвращаем уточненные углы

    # Если углы не найдены, отклоняем или щелкаем точки вручную
    else:
        if show:
            # Визуализатор и обработчик событий нажатия клавиш
            logging.info(f'{os.path.basename(img_path)}: Углы не найдены: пожалуйста, пометьте их вручную.')
            imgp_objp_confirmed = imgp_objp_visualizer_clicker(img, imgp=[], objp=objp, img_path=img_path)
        else:
            logging.info(f'{os.path.basename(img_path)}: Углы не найдены. Чтобы пометить их вручную, установите "show_detection_intrinsics" в true в файле Config.toml.')
            imgp_objp_confirmed = []

    return imgp_objp_confirmed  # Возвращаем подтвержденные углы

In [ ]:
def calibrate_intrinsics(calib_dir, intrinsics_config_dict):
    '''
    Рассчитать внутренние параметры
    на основе изображений или видео с шахматной доской.
    Извлечь кадры, затем обнаружить углы, затем выполнить калибровку.

    ВХОДНЫЕ ДАННЫЕ:
    - calib_dir: директория, содержащая папки с внутренними и внешними параметрами, каждая из которых заполнена директориями камер
    - intrinsics_config_dict: словарь параметров внутренней калибровки (overwrite_intrinsics, show_detection_intrinsics, intrinsics_extension, extract_every_N_sec, intrinsics_corners_nb, intrinsics_square_size, intrinsics_marker_size, intrinsics_aruco_dict)

    ВЫХОДНЫЕ ДАННЫЕ:
    - D: искажения: список массивов с плавающей точкой
    - K: внутренние параметры: список 3x3 массивов с плавающей точкой
    '''

    try:
        # Получаем список директорий камер из папки 'intrinsics'
        intrinsics_cam_listdirs_names = next(os.walk(os.path.join(calib_dir, 'intrinsics')))[1]
    except StopIteration:
        logging.exception(f'Ошибка: Папка {os.path.join(calib_dir, "intrinsics")} не найдена.')
        raise Exception(f'Ошибка: Папка {os.path.join(calib_dir, "intrinsics")} не найдена.')

    # Извлекаем параметры из словаря конфигурации
    intrinsics_extension = intrinsics_config_dict.get('intrinsics_extension')
    extract_every_N_sec = intrinsics_config_dict.get('extract_every_N_sec')
    overwrite_extraction = False
    show_detection_intrinsics = intrinsics_config_dict.get('show_detection_intrinsics')
    intrinsics_corners_nb = intrinsics_config_dict.get('intrinsics_corners_nb')
    intrinsics_square_size = intrinsics_config_dict.get('intrinsics_square_size') / 1000  # перевод в метры
    ret, C, S, D, K, R, T = [], [], [], [], [], [], []

    for i, cam in enumerate(intrinsics_cam_listdirs_names):
        # Подготовка объектных точек
        objp = np.zeros((intrinsics_corners_nb[0] * intrinsics_corners_nb[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0:intrinsics_corners_nb[0], 0:intrinsics_corners_nb[1]].T.reshape(-1, 2)
        objp[:, :2] = objp[:, 0:2] * intrinsics_square_size
        objpoints = []  # 3D точки в мировом пространстве
        imgpoints = []  # 2D точки в плоскости изображения

        logging.info(f'\nКамера {cam}:')
        img_vid_files = glob.glob(os.path.join(calib_dir, 'intrinsics', cam, f'*.{intrinsics_extension}'))
        if len(img_vid_files) == 0:
            logging.exception(f'Папка {os.path.join(calib_dir, "intrinsics", cam)} не существует или не содержит файлов с расширением .{intrinsics_extension}.')
            raise ValueError(f'Папка {os.path.join(calib_dir, "intrinsics", cam)} не существует или не содержит файлов с расширением .{intrinsics_extension}.')

        # Сортировка файлов по числовым значениям в имени
        img_vid_files = sorted(img_vid_files, key=lambda c: [int(n) for n in re.findall(r'\d+', c)])

        # Извлечение кадров из видео, если это видео
        try:
            cap = cv2.VideoCapture(img_vid_files[0])
            cap.read()
            if cap.read()[0] == False:
                raise
            extract_frames(img_vid_files[0], extract_every_N_sec, overwrite_extraction)
            img_vid_files = glob.glob(os.path.join(calib_dir, 'intrinsics', cam, f'*.png'))
            img_vid_files = sorted(img_vid_files, key=lambda c: [int(n) for n in re.findall(r'\d+', c)])
        except:
            pass

        # Поиск углов
        for img_path in img_vid_files:
            if show_detection_intrinsics == True:
                imgp_confirmed, objp_confirmed = findCorners(img_path, intrinsics_corners_nb, objp=objp, show=show_detection_intrinsics)
                if isinstance(imgp_confirmed, np.ndarray):
                    imgpoints.append(imgp_confirmed)
                    objpoints.append(objp_confirmed)
            else:
                imgp_confirmed = findCorners(img_path, intrinsics_corners_nb, objp=objp, show=show_detection_intrinsics)
                if isinstance(imgp_confirmed, np.ndarray):
                    imgpoints.append(imgp_confirmed)
                    objpoints.append(objp)

        if len(imgpoints) < 10:
            logging.info(f'Углы были обнаружены только на {len(imgpoints)} изображениях для камеры {cam}. Калибровка внутренних параметров может быть неточной при наличии менее 10 хороших изображений доски.')

        # Расчет внутренних параметров
        img = cv2.imread(str(img_path))
        objpoints = np.array(objpoints)
        ret_cam, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1],
                                    None, None, flags=(cv2.CALIB_FIX_K3))  # + cv2.CALIB_FIX_PRINCIPAL_POINT))
        h, w = [np.float32(i) for i in img.shape[:-1]]
        ret.append(ret_cam)
        C.append(cam)
        S.append([w, h])
        D.append(dist[0])
        K.append(mtx)
        R.append([0.0, 0.0, 0.0])
        T.append([0.0, 0.0, 0.0])

        logging.info(f'Ошибка внутренних параметров: {np.around(ret_cam, decimals=3)} пикселей для каждой камеры.')

    return ret, C, S, D, K, R, T

In [ ]:
#@title calibrate_extrinsics
def calibrate_extrinsics(calib_dir, extrinsics_config_dict, C, S, K, D):
    '''
    Калибрует экстраординарные параметры
    из изображения или первого кадра видео
    шахматной доски или измеренных объектов на сцене

    ВХОДНЫЕ ДАННЫЕ:
    - calib_dir: директория, содержащая папки с внутренними и экстраординарными параметрами, каждая из которых заполнена директориями камер
    - extrinsics_config_dict: словарь параметров экстраординарных параметров (extrinsics_method, calculate_extrinsics, show_detection_extrinsics, extrinsics_extension, extrinsics_corners_nb, extrinsics_square_size, extrinsics_marker_size, extrinsics_aruco_dict, object_coords_3d)

    ВЫХОДНЫЕ ДАННЫЕ:
    - R: экстраординарная ротация: список массивов с плавающей запятой (Родригес)
    - T: экстраординарный перевод: список массивов с плавающей запятой
    '''

    try:
        # Получаем список директорий камер из папки 'extrinsics'
        extrinsics_cam_listdirs_names = next(os.walk(os.path.join(calib_dir, 'extrinsics')))[1]
    except StopIteration:
        logging.exception(f'Ошибка: Папка {os.path.join(calib_dir, "extrinsics")} не найдена.')
        raise Exception(f'Ошибка: Папка {os.path.join(calib_dir, "extrinsics")} не найдена.')

    extrinsics_method = extrinsics_config_dict.get('extrinsics_method')
    ret, R, T = [], [], []

    if extrinsics_method in {'board', 'scene'}:

        # Определяем 3D координаты объектов
        if extrinsics_method == 'board':
            extrinsics_corners_nb = extrinsics_config_dict.get('board').get('extrinsics_corners_nb')
            extrinsics_square_size = extrinsics_config_dict.get('board').get('extrinsics_square_size') / 1000 # переводим в метры
            object_coords_3d = np.zeros((extrinsics_corners_nb[0] * extrinsics_corners_nb[1], 3), np.float32)
            object_coords_3d[:, :2] = np.mgrid[0:extrinsics_corners_nb[0], 0:extrinsics_corners_nb[1]].T.reshape(-1, 2)
            object_coords_3d[:, :2] = object_coords_3d[:, 0:2] * extrinsics_square_size
        elif extrinsics_method == 'scene':
            object_coords_3d = np.array(extrinsics_config_dict.get('scene').get('object_coords_3d'), np.float32)

        # Сохраняем 3D координаты объектов в формате trc
        calib_output_path = os.path.join(calib_dir, f'Object_points.trc')
        trc_write(object_coords_3d, calib_output_path)

        for i, cam in enumerate(extrinsics_cam_listdirs_names):
            logging.info(f'\nКамера {cam}:')

            # Читаем изображения или видео
            extrinsics_extension = [extrinsics_config_dict.get('board').get('extrinsics_extension') if extrinsics_method == 'board'
                                    else extrinsics_config_dict.get('scene').get('extrinsics_extension')][0]
            show_reprojection_error = [extrinsics_config_dict.get('board').get('show_reprojection_error') if extrinsics_method == 'board'
                                    else extrinsics_config_dict.get('scene').get('show_reprojection_error')][0]
            img_vid_files = glob.glob(os.path.join(calib_dir, 'extrinsics', cam, f'*.{extrinsics_extension}'))
            if len(img_vid_files) == 0:
                logging.exception(f'Папка {os.path.join(calib_dir, "extrinsics", cam)} не существует или не содержит файлов с расширением .{extrinsics_extension}.')
                raise ValueError(f'Папка {os.path.join(calib_dir, "extrinsics", cam)} не существует или не содержит файлов с расширением .{extrinsics_extension}.')
            img_vid_files = sorted(img_vid_files, key=lambda c: [int(n) for n in re.findall(r'\d+', c)]) # сортируем пути с числами

            # Извлекаем кадры из изображения или видео, если imread равно None
            img = cv2.imread(img_vid_files[0])
            if img is None:
                cap = cv2.VideoCapture(img_vid_files[0])
                res, img = cap.read()
                if res == False:
                    raise
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Находим углы или помечаем вручную
            if extrinsics_method == 'board':
                imgp = findCorners(img_vid_files[0], extrinsics_corners_nb, objp=object_coords_3d, show=show_reprojection_error)
                objp = object_coords_3d
                if len(imgp) == 0:
                    logging.exception('Углы не найдены. Установите "show_detection_extrinsics" в true, чтобы щелкнуть углы вручную, или измените extrinsic_board_type на "scene"')
                    raise ValueError('Углы не найдены. Установите "show_detection_extrinsics" в true, чтобы щелкнуть углы вручную, или измените extrinsic_board_type на "scene"')

            elif extrinsics_method == 'scene':
                imgp, objp = imgp_objp_visualizer_clicker(img, imgp=[], objp=object_coords_3d, img_path=img_vid_files[0])
                if len(imgp) == 0:
                    logging.exception('Не нажаты точки (или меньше 6). Нажмите \'C\', когда изображение отображается, а затем щелкните по точкам изображения, соответствующим \'object_coords_3d\', которые вы измерили и записали в файл Config.toml.')
                    raise ValueError('Не нажаты точки (или меньше 6). Нажмите \'C\', когда изображение отображается, а затем щелкните по точкам изображения, соответствующим \'object_coords_3d\', которые вы измерили и записали в файл Config.toml.')
                if len(objp) < 10:
                    logging.info(f'Только {len(objp)} контрольных точек для камеры {cam}. Калибровка экстраординарных параметров может быть неточной с менее чем 10 контрольными точками, распределенными по захваченному объему как можно шире.')

            elif extrinsics_method == 'keypoints':
                logging.info('Калибровка на основе ключевых точек пока не доступна.')

            # Вычисляем экстраординарные параметры
            mtx, dist = np.array(K[i]), np.array(D[i])
            _, r, t = cv2.solvePnP(np.array(objp)*1000, imgp, mtx, dist)
            r, t = r.flatten(), t.flatten()
            t /= 1000

            # Проекция 3D точек объектов на плоскость изображения
            proj_obj = np.squeeze(cv2.projectPoints(objp, r, t, mtx, dist)[0])

            # Проверка результатов калибровки
            if show_reprojection_error:
                # Снова открываем изображение, иначе два набора текста накладываются друг на друга
                img = cv2.imread(img_vid_files[0])
                if img is None:
                    cap = cv2.VideoCapture(img_vid_files[0])
                    res, img = cap.read()
                    if res == False:
                        raise
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                for o in proj_obj:
                    cv2.circle(img, (int(o[0]), int(o[1])), 8, (0,0,255), -1)
                for i in imgp:
                    cv2.drawMarker(img, (int(i[0][0]), int(i[0][1])), (0,255,0), cv2.MARKER_CROSS, 15, 2)
                cv2.putText(img, 'Проверьте результаты калибровки, затем закройте окно.', (20, 20), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
                cv2.putText(img, 'Проверьте результаты калибровки, затем закройте окно.', (20, 20), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)
                cv2.drawMarker(img, (20,40), (0,255,0), cv2.MARKER_CROSS, 15, 2)
                cv2.putText(img, '    Нажатые точки', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
                cv2.putText(img, '    Нажатые точки', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)
                cv2.circle(img, (20,60), 8, (0,0,255), -1)
                cv2.putText(img, '    Проецируемые точки объектов', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, .7, (255,255,255), 7, lineType = cv2.LINE_AA)
                cv2.putText(img, '    Проецируемые точки объектов', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, .7, (0,0,0), 2, lineType = cv2.LINE_AA)
                im_pil = Image.fromarray(img)
                im_pil.show(title = os.path.basename(img_vid_files[0]))

            # Вычисляем ошибку проекции
            imgp_to_objreproj_dist = [euclidean_distance(proj_obj[n], imgp[n]) for n in range(len(proj_obj))]
            rms_px = np.sqrt(np.sum([d**2 for d in imgp_to_objreproj_dist]))
            ret.append(rms_px)
            R.append(r)
            T.append(t)

    elif extrinsics_method == 'keypoints':
        raise NotImplementedError('Это еще не интегрировано.')

    else:
        raise ValueError('Неверное значение для extrinsics_method')

    return ret, C, S, D, K, R, T

In [ ]:
def calib_calc_fun(calib_dir, intrinsics_config_dict, extrinsics_config_dict):
    '''
    Calibrates intrinsic and extrinsic parameters
    from images or videos of a checkerboard
    or retrieve them from a file

    INPUTS:
    - calib_dir: directory containing intrinsic and extrinsic folders, each populated with camera directories
    - intrinsics_config_dict: dictionary of intrinsics parameters (overwrite_intrinsics, show_detection_intrinsics, intrinsics_extension, extract_every_N_sec, intrinsics_corners_nb, intrinsics_square_size, intrinsics_marker_size, intrinsics_aruco_dict)
    - extrinsics_config_dict: dictionary of extrinsics parameters (calculate_extrinsics, show_detection_extrinsics, extrinsics_extension, extrinsics_corners_nb, extrinsics_square_size, extrinsics_marker_size, extrinsics_aruco_dict, object_coords_3d)

    OUTPUTS:
    - ret: residual reprojection error in _px_: list of floats
    - C: camera name: list of strings
    - S: image size: list of list of floats
    - D: distorsion: list of arrays of floats
    - K: intrinsic parameters: list of 3x3 arrays of floats
    - R: extrinsic rotation: list of arrays of floats (Rodrigues)
    - T: extrinsic translation: list of arrays of floats
    '''

    overwrite_intrinsics = intrinsics_config_dict.get('overwrite_intrinsics')
    calculate_extrinsics = extrinsics_config_dict.get('calculate_extrinsics')

    # retrieve intrinsics if calib_file found and if overwrite_intrinsics=False
    try:
        calib_file = glob.glob(os.path.join(calib_dir, f'Calib*.toml'))[0]
    except:
        pass
    if not overwrite_intrinsics and 'calib_file' in locals():
        logging.info(f'\nPreexisting calibration file found: \'{calib_file}\'.')
        logging.info(f'\nRetrieving intrinsic parameters from file. Set "overwrite_intrinsics" to true in Config.toml to recalculate them.')
        calib_file = glob.glob(os.path.join(calib_dir, f'Calib*.toml'))[0]
        calib_data = toml.load(calib_file)

        ret, C, S, D, K, R, T = [], [], [], [], [], [], []
        for cam in calib_data:
            if cam != 'metadata':
                ret += [0.0]
                C += [calib_data[cam]['name']]
                S += [calib_data[cam]['size']]
                K += [np.array(calib_data[cam]['matrix'])]
                D += [calib_data[cam]['distortions']]
                R += [[0.0, 0.0, 0.0]]
                T += [[0.0, 0.0, 0.0]]
        nb_cams_intrinsics = len(C)

    # calculate intrinsics otherwise
    else:
        logging.info(f'\nCalculating intrinsic parameters...')
        ret, C, S, D, K, R, T = calibrate_intrinsics(calib_dir, intrinsics_config_dict)
        nb_cams_intrinsics = len(C)

    # calculate extrinsics
    if calculate_extrinsics:
        logging.info(f'\nCalculating extrinsic parameters...')

        # check that the number of cameras is consistent
        nb_cams_extrinsics = len(next(os.walk(os.path.join(calib_dir, 'extrinsics')))[1])
        if nb_cams_intrinsics != nb_cams_extrinsics:
            raise Exception(f'Error: The number of cameras is not consistent:\
                    Found {nb_cams_intrinsics} cameras based on the number of intrinsic folders or on calibration file data,\
                    and {nb_cams_extrinsics} cameras based on the number of extrinsic folders.')
        ret, C, S, D, K, R, T = calibrate_extrinsics(calib_dir, extrinsics_config_dict, C, S, K, D)
    else:
        logging.info(f'\nExtrinsic parameters won\'t be calculated. Set "calculate_extrinsics" to true in Config.toml to calculate them.')

    return ret, C, S, D, K, R, T

In [ ]:
def calibrate_cams_all(config_dict):
    '''
    Либо конвертирует существующий файл калибровки,
    либо вычисляет калибровку с нуля (с доски или из точек).
    Сохраняет калибровку в .toml файл.
    Печатает сводку.

    ВХОДНЫЕ ДАННЫЕ:
    - словарь config_dict

    ВЫХОД:
    - файл калибровки камеры в формате .toml
    '''

    # Чтение config_dict
    project_dir = config_dict.get('project').get('project_dir')  # Получаем директорию проекта
    # Находим директорию калибровки
    calib_dir = [os.path.join(project_dir, c) for c in os.listdir(project_dir) if ('Calib' in c or 'calib' in c)][0]
    calib_type = config_dict.get('calibration').get('calibration_type')  # Получаем тип калибровки

    if calib_type == 'convert':
        convert_filetype = config_dict.get('calibration').get('convert').get('convert_from')  # Получаем тип файла для конвертации
        try:
            if convert_filetype == 'qualisys':
                convert_ext = '.qca.txt'  # Расширение для Qualisys
                file_to_convert_path = glob.glob(os.path.join(calib_dir, f'*{convert_ext}*'))[0]  # Находим файл для конвертации
                binning_factor = config_dict.get('calibration').get('convert').get('qualisys').get('binning_factor')  # Получаем фактор биннинга
            elif convert_filetype == 'optitrack':
                file_to_convert_path = ['']  # Пустой путь для OptiTrack
                binning_factor = 1
            elif convert_filetype == 'vicon':
                convert_ext = '.xcp'  # Расширение для Vicon
                file_to_convert_path = glob.glob(os.path.join(calib_dir, f'*{convert_ext}'))[0]
                binning_factor = 1
            elif convert_filetype == 'opencap':  # Все файлы с расширением .pickle
                convert_ext = '.pickle'
                file_to_convert_path = sorted(glob.glob(os.path.join(calib_dir, f'*{convert_ext}')))
                binning_factor = 1
            elif convert_filetype == 'easymocap':  # Файлы intri.yml
                convert_ext = '.yml'
                file_to_convert_path = sorted(glob.glob(os.path.join(calib_dir, f'*{convert_ext}')))
                binning_factor = 1
            elif convert_filetype == 'biocv':  # Все файлы без расширения -> теперь с расширением .calib
                convert_ext = '.calib'
                file_to_convert_path = sorted(glob.glob(os.path.join(calib_dir, f'*{convert_ext}')))
                binning_factor = 1
            elif convert_filetype in ['anipose', 'freemocap', 'caliscope']:  # Конвертация не требуется, пропускаем этот этап
                logging.info(f'\n--> Конвертация из Caliscope, AniPose или FreeMocap не требуется. Калибровка пропущена.\n')
                return
            else:
                convert_ext = '???'
                file_to_convert_path = ['']
                raise NameError(f'Конвертация калибровки из {convert_filetype} не поддерживается.') from None
            assert file_to_convert_path != []
        except:
            raise NameError(f'Файл с расширением {convert_ext} не найден в {calib_dir}.')

        calib_output_path = os.path.join(calib_dir, f'Calib_{convert_filetype}.toml')  # Путь для сохранения файла калибровки
        calib_full_type = '_'.join([calib_type, convert_filetype])  # Полный тип калибровки
        args_calib_fun = [file_to_convert_path, binning_factor]  # Аргументы для функции калибровки

    elif calib_type == 'calculate':
        intrinsics_config_dict = config_dict.get('calibration').get('calculate').get('intrinsics')  # Получаем параметры внутренней калибровки
        extrinsics_config_dict = config_dict.get('calibration').get('calculate').get('extrinsics')  # Получаем параметры внешней калибровки
        extrinsics_method = config_dict.get('calibration').get('calculate').get('extrinsics').get('extrinsics_method')  # Получаем метод внешней калибровки

        calib_output_path = os.path.join(calib_dir, f'Calib_{extrinsics_method}.toml')  # Путь для сохранения файла калибровки
        calib_full_type = calib_type  # Полный тип калибровки
        args_calib_fun = [calib_dir, intrinsics_config_dict, extrinsics_config_dict]  # Аргументы для функции калибровки

    else:
        logging.info('Неверный тип калибровки в Config.toml')

    # Сопоставление функции калибровки
    '''

    calib_mapping = {
        'convert_qualisys': calib_qca_fun,
        'convert_optitrack': calib_optitrack_fun,
        'convert_vicon': calib_vicon_fun,
        'convert_opencap': calib_opencap_fun,
        'convert_easymocap': calib_easymocap_fun,
        'convert_biocv': calib_biocv_fun,
        'calculate': calib_calc_fun,
    }
    '''
    calib_mapping = {
        'calculate': calib_calc_fun,
    }
    calib_fun = calib_mapping[calib_full_type]  # Получаем соответствующую функцию калибровки

    # Выполнение калибровки
    ret, C, S, D, K, R, T = calib_fun(*args_calib_fun)

    # Запись файла калибровки
    toml_write(calib_output_path, C, S, D, K, R, T)

    # Сообщение о результатах
    recap_calibrate(ret, calib_output_path, calib_full_type)

In [ ]:
def calibration(config=None):
    '''
    Калибровка камер с использованием шахматных досок или файлов Qualisys.

    config может быть словарем,
    или путем к директории испытания, участника или сессии,
    или функция может быть вызвана без аргумента, в этом случае директория конфигурации будет текущей.
    '''

    #from Pose2Sim.calibration import calibrate_cams_all

    # Чтение конфигурационных файлов и определение уровня
    level, config_dicts = read_config_files(config, config_dir=config_dir)
    config_dict = config_dicts[0]

    try:
        # Устанавливаем директорию сессии равной директории конфигурации
        session_dir = config_dir

        print('session_dir', session_dir)
        # Поиск директории калибровки
        [os.path.join(session_dir, c) for c in os.listdir(session_dir) if 'calib' in c.lower() and not c.lower().endswith('.py')][0]
    except:
        # Если не удалось найти директорию, используем текущую
        session_dir = os.path.realpath(os.getcwd())

    # Обновление словаря проекта с директорией сессии
    config_dict.get("project").update({"project_dir": session_dir})

    # Настройка логирования
    #setup_logging(session_dir)
    currentDateAndTime = datetime.now()

    # Запуск калибровки
    # Поиск директории калибровки
    print('session_dir', session_dir)
    calib_dir = [os.path.join(session_dir, c) for c in os.listdir(session_dir) if os.path.isdir(os.path.join(session_dir, c)) and 'calib' in c.lower()][0]

    # Логирование информации о калибровке
    logging.info("\n---------------------------------------------------------------------")
    logging.info("Калибровка камер")
    logging.info(f"Дата и время: {currentDateAndTime.strftime('%A %d. %B %Y, %H:%M:%S')}")
    logging.info(f"Директория калибровки: {calib_dir}")
    logging.info("---------------------------------------------------------------------\n")

    # Запись времени начала калибровки
    start = time.time()

    # Выполнение калибровки камер
    calibrate_cams_all(config_dict)

    # Запись времени окончания калибровки
    end = time.time()
    # Логирование времени, затраченного на калибровку
    logging.info(f'\nКалибровка заняла {end-start:.2f} секунд.\n')

In [ ]:
calibration()

session_dir /content/pose2sim/Pose2Sim/Demo_SinglePerson/
session_dir /content/pose2sim/Pose2Sim/Demo_SinglePerson/


KeyError: 'convert_qualisys'